##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Создание оценщика из модели Keras

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/keras_model_to_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/estimator/keras_model_to_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

## Введение

Эстиматоры(оценщики) полностью поддерживаются в TensorFlow и могут быть созданы из новых и существующих моделей `tf.keras`. Это руководство содержит полный минимальный пример этого процесса.

## Установка

In [ ]:
import tensorflow as tf

import numpy as np
import tensorflow_datasets as tfds

### Создание простой модели Keras.

В Keras вы собираете *слои* для построения *моделей*. Модель - это (обычно) граф
слоев. Самый распространенный тип модели - это стек слоев: модель `tf.keras.Sequential`.

Чтобы построить простую полносвязанную сеть(например, многослойный перцептрон):

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3)
])

Скомпилируйте модель и посмотрите ее архитектуру с помощью метода `summary()`

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam')
model.summary()

### Создание функции ввода

Используйте [Datasets API](../../guide/data.md) для масштабирования больших наборов данных
или обучения на нескольких устройствах.

Оценщикам необходимо контролировать, когда и как строится их входной конвейер. Для этого им требуется «функция ввода» или `input_fn`. `Estimator` вызовет эту функцию без аргументов. `input_fn` должна возвращать объект `tf.data.Dataset`.

In [ ]:
def input_fn():
  split = tfds.Split.TRAIN
  dataset = tfds.load('iris', split=split, as_supervised=True)
  dataset = dataset.map(lambda features, labels: ({'dense_input':features}, labels))
  dataset = dataset.batch(32).repeat()
  return dataset

Протестируйте `input_fn`

In [ ]:
for features_batch, labels_batch in input_fn().take(1):
  print(features_batch)
  print(labels_batch)

### Создание эстиматора из модели tf.keras.

`tf.keras.Model` можно обучить с помощью `tf.estimator` путем преобразования модели в объект `tf.estimator.Estimator` с помощью метода `tf.keras.estimator.model_to_estimator`.

In [ ]:
import tempfile
model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model, model_dir=model_dir)

Обучите и проверьте ваш эстиматор.

In [ ]:
keras_estimator.train(input_fn=input_fn, steps=500)
eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10)
print('Eval result: {}'.format(eval_result))